# TODO

## Major
- [x] create term-by-document matrix (calculate words frequncies for each term-document pair)
 - [ ] check that it's actually correct - especially if we don't map terms to wrong documents
- [x] convert term-by-document frequencies to tf-idf (calcualte tf-idf for each term-document pair)
 - [ ] check
- [ ] we may need actual (numpy?) matrix?
- [ ] LSI magic

### Minor
- [x] remove numbers from terms - done but not sure if it's good thing to do, maybe it's also important for relevancy of docs,
like for example when there is year written?

In [1]:
import pandas as pd
import numpy as np
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
np.random.seed(42)

In [3]:
bp_data = pd.read_csv("articles.csv", header=0)

In [4]:
bp_data.head(1)

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."


In [5]:
def preprocess_docs(docs, use_lemmatizer = True):
    '''Tokenize and preprocess documents
    
    Parameters
    ----------
    use_lemmatizer : bool
                     Uses lemmazizer if True, othrerwise uses stemmer.
    '''
    preproccessed_docs = []
    
    # English stop words list
    en_stop = set(stopwords.words('english'))
    
    # Word tokenizer that removes punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    
    # lemmatizer / Stemmer
    if use_lemmatizer:
        lemmatizer = WordNetLemmatizer()
    else:
        stemmer = SnowballStemmer("english")
    
    for row in docs.itertuples(index=True, name='Doc'):
        text = row.text
        
        # remove numbers
        text = re.sub(r'\d+', '', text)
        
        text_words = tokenizer.tokenize(text)
        
        if use_lemmatizer:
            text_words = [lemmatizer.lemmatize(word, pos="v").lower() for word in text_words
                          if word not in string.punctuation and word.lower() not in en_stop]
        else:
            text_words = [stemmer.stem(word).lower() for word in text_words
                         if word not in string.punctuation and word.lower() not in en_stop]
        
        preproccessed_docs.append({'words': text_words})
    
    pdocs = pd.DataFrame(preproccessed_docs)
    return pdocs

In [6]:
preproccessed_docs = preprocess_docs(bp_data)
display(preproccessed_docs)

,words
0,"[oh, headline, blare, chatbots, next, big, thi..."
1,"[ever, find, look, question, concept, syntax, ..."
2,"[machine, learn, increasingly, move, hand, des..."
3,"[understand, machine, learning, big, question,..."
4,"[want, learn, apply, artificial, intelligence,..."
...,...
332,"[click, share, article, linkedin, skip, part, ..."
333,"[opinions, deep, neural, network, machine, lea..."
334,"[everyone, remotely, tune, recent, progress, m..."
335,"[one, biggest, misconceptions, around, idea, d..."


In [9]:
def get_term_by_document_frequency(preprocessed_docs):
    document_by_term = {}
    
    for index, row in preprocessed_docs.iterrows():
        doc_id = index
        doc_words = row['words']
        
        # computed later, @TODO: move computation here and fix below or remove
#         document_by_term[doc_id] = {
#             'total_words': len(doc_words)
#         }
        document_by_term[doc_id] = {}
        
        for word in set(row['words']):
            document_by_term[doc_id][word] = doc_words.count(word)

    df = pd.DataFrame(document_by_term)
    
    return df

In [10]:
df_frequency = get_term_by_document_frequency(preproccessed_docs)

In [11]:
df_frequency

,0,1,2,3,4,5,6,7,8,9,...,327,328,329,330,331,332,333,334,335,336
wait,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
fair,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
request,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
behind,1.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,4.0,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
speech,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
paradoxes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
initialisation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
intelligible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
illusion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [14]:
def reduce_terms(df_frequency, max_df=1, min_df=0, max_terms=None):
    '''Remove unimportant terms from term-by-document matrix.
    
    Parameters
    ----------
    df : pd.DataFrame
    max_df : float , between [0, 1]
             Terms that appear in more % of documents will be ignored
    min_df : float , between [0, 1]
             Terms that appear in less % of documents will be ignored
    max_terms : int , None
                If not None, only top `max_terms` terms will be returned.
    '''
    df = df_frequency.copy()
    if 'doc_frequency' in df:
        df = df.drop(columns='doc_frequency')
    
    corpus_size = df.shape[1]
    
    df['doc_frequency'] = df_frequency.fillna(0).astype(bool).sum(axis=1) / corpus_size
    
    df = df[df.doc_frequency <= max_df]
    df = df[df.doc_frequency >= min_df]
    
    if max_terms is not None:
        assert('not implementd' == False) # @TODO - implement or remove
    
    return df

In [15]:
reduce_terms(df_frequency).sort_values('doc_frequency', ascending=False).shape

(16603, 338)

In [16]:
reduce_terms(df_frequency, 0.8, 0.1).sort_values('doc_frequency', ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,328,329,330,331,332,333,334,335,336,doc_frequency
take,7.0,3.0,4.0,5.0,9.0,2.0,NaN,4.0,NaN,5.0,...,5.0,2.0,NaN,6.0,2.0,1.0,4.0,NaN,3.0,0.777448
way,9.0,1.0,1.0,2.0,4.0,4.0,2.0,2.0,2.0,9.0,...,NaN,11.0,NaN,5.0,1.0,1.0,NaN,1.0,7.0,0.765579
find,NaN,3.0,2.0,3.0,1.0,3.0,1.0,2.0,1.0,7.0,...,3.0,5.0,NaN,4.0,3.0,2.0,NaN,NaN,5.0,0.762611
machine,6.0,2.0,10.0,11.0,2.0,NaN,20.0,3.0,2.0,5.0,...,2.0,8.0,2.0,1.0,6.0,11.0,4.0,4.0,NaN,0.762611
need,3.0,1.0,5.0,2.0,6.0,3.0,1.0,NaN,NaN,7.0,...,9.0,7.0,6.0,6.0,6.0,10.0,1.0,3.0,4.0,0.756677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,0.100890
unsupervised,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,0.100890
ideal,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100890
encourage,NaN,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.100890


In [17]:
df_reduced = reduce_terms(df_frequency, 0.8, 0.1)

In [19]:
def get_tf_idf(df_frequency):
    df = df_frequency.copy()
    # tf := word frequency / total frequency
    df.loc['total_words'] = df.sum()
    
    df = df.drop('total_words')[:] / df.loc['total_words']
    
    # idf := log ( len(all_documents) / len(documents_containing_word) )
    
    corpus_size = df.shape[1]

    # number of non-zero cols
    df['doc_frequency'] = df.fillna(0).astype(bool).sum(axis=1)
        
    df['idf'] = np.log( corpus_size / df['doc_frequency'] )
    
    # tf-idf := tf * idf
    _cols = df.columns.difference(['idf', 'doc_frequency'])
    df[_cols] = df[_cols].multiply(df["idf"], axis="index")
    
    df.drop(columns=['doc_frequency', 'idf'], inplace=True)
    
    return df

In [20]:
df_tf_idf = get_tf_idf(df_reduced)
display(df_tf_idf)

,0,1,2,3,4,5,6,7,8,9,...,327,328,329,330,331,332,333,334,335,336
wait,0.001356,NaN,NaN,NaN,0.001116,NaN,NaN,NaN,NaN,NaN,...,0.000502,NaN,0.001366,NaN,NaN,0.001665,NaN,NaN,NaN,NaN
fair,0.001535,NaN,NaN,0.002239,NaN,NaN,NaN,NaN,0.003436,NaN,...,0.001705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001543
behind,0.000953,NaN,0.000855,NaN,0.001569,NaN,NaN,NaN,NaN,0.002479,...,0.000353,NaN,NaN,0.002651,NaN,NaN,NaN,NaN,NaN,NaN
speech,0.002525,NaN,NaN,NaN,NaN,NaN,0.002311,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.001261,NaN,NaN,NaN
come,0.000739,0.000771,0.000221,0.001078,0.000203,0.000911,0.000451,0.002076,0.001655,0.000961,...,0.000547,NaN,0.000248,NaN,0.00062,0.000907,NaN,NaN,NaN,0.000495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neuron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000542,NaN,NaN,NaN,NaN,NaN,NaN,0.011939,0.028602,0.008836
behavior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004026,0.015782,NaN
improvements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.003168,NaN,NaN,NaN,NaN,NaN,0.005585,NaN


In [21]:
values = df_tf_idf.fillna(0).to_numpy()
values

array([[0.0013562 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00153473, 0.        , 0.        , ..., 0.        , 0.        ,
        0.0015434 ],
       [0.00095305, 0.        , 0.00085462, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00402571, 0.0157816 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00558486,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [23]:
u, s, vh = np.linalg.svd(values, full_matrices=True)

In [26]:
vh

array([[-4.38027008e-02, -4.52527257e-02, -5.49950587e-02, ...,
        -5.03892281e-02, -5.13812996e-02, -5.13567737e-02],
       [ 1.20471112e-02,  9.93265786e-03,  8.84456061e-03, ...,
         1.38319783e-02,  1.34522571e-02,  1.30991734e-02],
       [-4.57748185e-03, -7.51473102e-03, -1.46184934e-02, ...,
        -1.57943086e-02, -1.00991313e-02, -1.59776393e-02],
       ...,
       [ 0.00000000e+00, -1.83744577e-17, -3.12236288e-17, ...,
        -3.51281504e-17, -3.46944695e-18, -5.89805982e-17],
       [ 0.00000000e+00, -2.78299028e-16, -1.27006870e-16, ...,
         2.42861287e-17,  6.93889390e-18,  5.81132364e-17],
       [ 8.16481618e-01,  1.94289029e-16, -3.88578059e-16, ...,
         7.25060203e-18, -3.83570400e-17, -4.23245423e-17]])